### Evaluate model

In [ ]:
import matplotlib.pyplot as plt
import torch

from datasets import load_dataset
from torchvision.transforms import ToPILImage
from torchvision.ops import box_iou
from transformers import DetrForObjectDetection, DetrImageProcessor

from Cocordiais import CocordiaisDataset, CocordiaisUtils
from PIL import Image as PImage

In [ ]:
HF_DATASET = "thiagohersan/cordiais-faces"
DETR_MODEL = "facebook/detr-resnet-50"

detr_size = { "shortest_edge": 800, "longest_edge": 800 }
detr_processor = DetrImageProcessor.from_pretrained(DETR_MODEL, size=detr_size)

hf_dataset = load_dataset(HF_DATASET)
hf_dataset_train = hf_dataset["train"].train_test_split(test_size=0.2, shuffle=True, seed=101010)

dataset = {
  "eval": {
    "images": hf_dataset_train["test"]["image"],
    "data": CocordiaisDataset(hf_dataset_train["test"], img_processor=detr_processor, train=False).data
  },
  "test": {
    "images": hf_dataset["test"]["image"],
    "data": CocordiaisDataset(hf_dataset["test"], img_processor=detr_processor, train=False).data
  }
}

print(
  f"Number of examples:\n"
  f"  Evaluation: {len(dataset['eval']['data'])}\n"
  f"  Test: {len(dataset['test']['data'])}"
)

In [ ]:
def get_bboxes_xyxy(labels):
  ih, iw = tuple(labels["orig_size"])
  bboxes_xyxy = []
  for (xc, yc, bw, bh) in labels["boxes"]:
    bboxes_xyxy.append([
      (xc - bw / 2) * iw,
      (yc - bh / 2) * ih,
      (xc + bw / 2) * iw,
      (yc + bh / 2) * ih
    ])
  return torch.tensor(bboxes_xyxy)

def top_label_error(expected, estimated):
  if len(estimated["scores"]) < 1:
    return 1

  top_score_idx = estimated["scores"].argmax()
  top_score_label = estimated["labels"][top_score_idx]
  top_score_box = estimated["boxes"][top_score_idx]

  biou = box_iou(top_score_box.unsqueeze(0), expected["boxes"])
  biou_max_idx = biou.argmax()
  biou_max_label = expected["labels"][biou_max_idx]

  return (top_score_label != biou_max_label).sum().item()

In [ ]:
PLOT_COLORS = [
  [0.494, 0.184, 0.556], [0.929, 0.694, 0.125]
]

def plot_results(pil_img, results):
  scores = results["scores"]
  labels = results["labels"]
  boxes = results["boxes"]

  plt.figure(figsize=(16,10))
  plt.imshow(pil_img)
  ax = plt.gca()

  for score, label, (xmin, ymin, xmax, ymax) in zip(scores.tolist(), labels.tolist(), boxes.tolist()):
    ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color=PLOT_COLORS[label], linewidth=3))
    text = f'{CocordiaisUtils.ID2LABEL[label]}: {score:0.2f}'
    ax.text(xmin, ymin, text, fontsize=15, bbox=dict(facecolor='yellow', alpha=0.5))
  plt.axis('off')
  plt.show()

In [ ]:
MODEL_NAMES = [
  "detr-cordiais-no-aug-48",
  "detr-cordiais-aug-48",
  "detr-cordiais-no-aug-64",
  "detr-cordiais-aug-64",
  "detr-cordiais-no-aug-100",
  "detr-cordiais-aug-100",
  "detr-cordiais-aug1-100",
  "detr-cordiais-aug1-100-no_norm"
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
metrics = []

for model_name in MODEL_NAMES:
  hf_model = f"thiagohersan/{model_name}"
  detr_model = DetrForObjectDetection.from_pretrained(hf_model, id2label=CocordiaisUtils.ID2LABEL)
  detr_model = detr_model.to(device)

  for split in ["eval", "test"]:
    model_eval = {}

    for data_idx, data in enumerate(dataset[split]["data"]):
      pixel_values, pixel_mask, labels = data.values()
      pixel_values = pixel_values.unsqueeze(0).to(device)
      pixel_mask = pixel_mask.unsqueeze(0).to(device)
      labels = [{k: v.to(device) for k, v in labels.items()}]

      with torch.no_grad():
        outputs = detr_model(pixel_values=pixel_values, pixel_mask=pixel_mask, labels=labels)

      loss = outputs.loss
      loss_dict = outputs.loss_dict

      model_eval["loss"] = model_eval.get("loss", 0) + loss.item()
      for k,v in loss_dict.items():
        model_eval[k] = model_eval.get(k, 0) + v.item()

      orig_h, orig_w = labels[0]["orig_size"].tolist()
      estimated = detr_processor.post_process_object_detection(
        outputs,
        target_sizes=[(orig_h, orig_w)],
        threshold=0.5
      )

      expected = [{
        "labels": labels[0]["class_labels"],
        "boxes": get_bboxes_xyxy(labels[0])
      }]

      model_eval["label_error"] = model_eval.get("label_error", 0) + top_label_error(expected[0], estimated[0])
      model_eval["samples"] = model_eval.get("samples", 0) + 1

      # image = ToPILImage()(pixel_values.squeeze())
      # image = dataset[split]["images"][data_idx]
      # plot_results(image, estimated[0])
      # plot_results(image, expected[0])

    for k, v in model_eval.items():
      if k != "samples":
        model_eval[k] = round(v / model_eval["samples"], 4)
    model_eval["split"] = split
    model_eval["model"] = model_name.replace("detr-cordiais-", "")

    metrics.append(model_eval)

In [ ]:
import csv

fieldnames = list(metrics[0].keys())

with open('metrics.csv', 'w', newline='') as metrics_csv:
  writer = csv.DictWriter(metrics_csv, fieldnames=fieldnames)
  writer.writeheader()
  for m in metrics:
    writer.writerow(m)